In [2]:
!pip install vaderSentiment

     -------------------------------------- 126.0/126.0 kB 3.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import re
analyser = SentimentIntensityAnalyzer()

In [5]:
data=pd.read_csv("tshirt-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]


In [6]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()


,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/ftx-solid-men-round-neck-m...,0,Pack of 4 Solid Men Round Neck Multicolor T-Shirt,4.2,1103,147,"['Product quantity is excellent', 'Nyc look th..."
1,http://flipkart.com/louis-philippe-sport-solid...,1,Solid Men Polo Neck Light Blue T-Shirt,4.1,14974,1286,['The colour is just an awesome combination of...
2,http://flipkart.com/eyebogler-striped-men-polo...,2,"Striped Men Polo Neck White, Blue, Yellow T-Shirt",4.1,15561,1650,['Good exactly same as in the catalog. Fabric ...
3,http://flipkart.com/eyebogler-color-block-men-...,3,Color Block Men Round Neck Grey T-Shirt,3.9,88178,7932,"['Good product', 'Nice t-shirt... 👌', 'Awsome ..."
4,http://flipkart.com/tripr-printed-men-round-ne...,4,Printed Men Round Neck Pink T-Shirt,3.8,15858,1238,"['Perfect 🥰 💞', 'Osm color & design but improv..."


In [7]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_15048/2131956584.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [8]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
#     print(x)
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_15048/2037979432.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vader_score'][ind] =sum_score


In [9]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()
print(total_star_givers)
print(total_review_givers)


6408214
683250


In [10]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)


In [11]:
data.head()


,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/ftx-solid-men-round-neck-m...,0,Pack of 4 Solid Men Round Neck Multicolor T-Shirt,4.2,1103,147,product quantity is excellent nyc look this cl...,1.1968,0.000980
1,http://flipkart.com/louis-philippe-sport-solid...,1,Solid Men Polo Neck Light Blue T-Shirt,4.1,14974,1286,the colour is just an awesome combination of m...,3.0547,0.015330
2,http://flipkart.com/eyebogler-striped-men-polo...,2,"Striped Men Polo Neck White, Blue, Yellow T-Shirt",4.1,15561,1650,good exactly same as in the catalog fabric sof...,3.5025,0.018414
3,http://flipkart.com/eyebogler-color-block-men-...,3,Color Block Men Round Neck Grey T-Shirt,3.9,88178,7932,good product nice tshirt 👌 awsome shirtthanks ...,1.5916,0.072142
4,http://flipkart.com/tripr-printed-men-round-ne...,4,Printed Men Round Neck Pink T-Shirt,3.8,15858,1238,perfect 🥰 💞 osm color design but improvement ...,2.9271,0.014707


In [12]:
data.to_csv('tshirts_csv_final.csv')


In [13]:
Dict_pattern = {} 
Dict_neck={}
Dict_colour={}
item_pattern={}
item_neck={}
item_colour={}
for ind in data.index:
    men_index=0
    neck_index=0
    tshirt_index=0
    d=data['desc'][ind]
    d=d.lower()
    arr=d.split()
    men_index=arr.index("men")
    if "neck" in arr:
        neck_index=arr.index("neck")
    elif "v-neck" in arr:
        neck_index=arr.index("v-neck")
    elif "collar" in arr:
        neck_index=arr.index("collar")
    elif "crew" in arr:
        neck_index=arr.index("crew")
    tshirt_index=arr.index("t-shirt")
    pattern1=arr[0:men_index]
    neck1=arr[men_index+1:neck_index+1]
    colour1=arr[neck_index+1:tshirt_index+1]
    pattern2=" ".join(pattern1)
    neck2=" ".join(neck1)
    colour2=" ".join(colour1)
    pattern2= re.sub(r'[^\w\s]', '', pattern2) 
    neck2= re.sub(r'[^\w\s]', '', neck2) 
    colour2= re.sub(r'[^\w\s]', '', colour2) 
    if pattern2 in Dict_pattern:
        Dict_pattern[pattern2]=Dict_pattern[pattern2]+1
        item_pattern[pattern2].append(ind)
    else:
        Dict_pattern[pattern2]=1
        item_pattern[pattern2]=[]
        item_pattern[pattern2].append(ind)
        
    if neck2 in Dict_neck:
        Dict_neck[neck2]=Dict_neck[neck2]+1
        item_neck[neck2].append(ind)
    else:
        Dict_neck[neck2]=1
        item_neck[neck2]=[]
        item_neck[neck2].append(ind)
        
    if colour2 in Dict_colour:
        Dict_colour[colour2]=Dict_colour[colour2]+1
        item_colour[colour2].append(ind)
    else:
        Dict_colour[colour2]=1
        item_colour[colour2]=[]
        item_colour[colour2].append(ind)

In [14]:
print_sorted= sorted(Dict_pattern, key=Dict_pattern.get, reverse=True)
for r in print_sorted:
    print(r, Dict_pattern[r])

solid 153
printed 67
striped 48
color block 48
pack of 3 solid 15
pack of 4 solid 12
pack of 2 solid 8
typography 7
graphic print 7
pack of 2 striped 6
self design 4
pack of 2 color block 3
pack of 2 self design 2
pack of 3 striped 1
military camouflage 1
checkered 1
ombre 1
striped applique 1
pack of 3 printed 1
pack of 3 color block 1


In [15]:
row=print_sorted[0:5]+print_sorted[-5:]  
import csv
with open('print_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_pattern[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)


[['solid', 1, 7, 10, 11, 12, 13, 16, 18, 19, 24, 31, 33, 35, 37, 39, 44, 45, 47, 48, 51, 52, 53, 55, 58, 59, 68, 78, 81, 83, 87, 90, 94, 95, 100, 104, 106, 107, 109, 121, 125, 126, 127, 128, 130, 131, 135, 136, 138, 139, 140, 144, 145, 146, 147, 150, 153, 154, 156, 157, 162, 165, 167, 172, 175, 176, 182, 183, 184, 185, 191, 192, 194, 199, 201, 202, 203, 205, 208, 209, 211, 212, 213, 217, 218, 220, 221, 223, 224, 225, 227, 228, 230, 231, 237, 238, 240, 241, 243, 244, 247, 250, 251, 253, 255, 256, 257, 262, 266, 267, 268, 275, 277, 278, 282, 284, 285, 289, 293, 296, 297, 300, 303, 304, 307, 308, 311, 314, 320, 324, 328, 329, 331, 332, 338, 339, 343, 344, 345, 346, 348, 350, 354, 359, 362, 364, 365, 370, 374, 375, 378, 379, 381, 382], ['printed', 4, 8, 14, 17, 36, 49, 69, 70, 80, 82, 84, 91, 96, 110, 114, 122, 134, 158, 160, 161, 166, 169, 186, 196, 198, 204, 207, 214, 219, 226, 229, 233, 235, 246, 249, 270, 274, 276, 279, 280, 283, 288, 290, 292, 294, 298, 302, 309, 310, 313, 317, 321, 3

In [16]:
neck_sorted= sorted(Dict_neck, key=Dict_neck.get, reverse=True)
for r in neck_sorted:
    print(r, Dict_neck[r])

round neck 196
polo neck 129
hooded neck 29
mandarin collar 17
henley neck 4
v neck 4
collared neck 2
round or crew 2
crew neck 2
high neck 1
scoop neck 1


In [17]:
row=neck_sorted[0:5]+neck_sorted[-5:]   
import csv
with open('neck_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_neck[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['round neck', 0, 3, 4, 8, 14, 15, 17, 20, 21, 23, 24, 25, 27, 28, 29, 30, 32, 34, 37, 38, 41, 42, 47, 49, 50, 56, 58, 59, 60, 61, 63, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 79, 80, 81, 84, 85, 86, 88, 89, 90, 91, 92, 93, 97, 98, 100, 105, 106, 107, 110, 111, 112, 113, 115, 117, 118, 119, 120, 122, 123, 124, 128, 129, 132, 137, 138, 143, 144, 147, 148, 149, 151, 154, 157, 158, 159, 160, 161, 163, 166, 168, 169, 170, 171, 174, 177, 178, 181, 184, 186, 189, 190, 191, 192, 196, 197, 198, 200, 203, 204, 215, 216, 222, 223, 228, 229, 230, 232, 233, 236, 239, 242, 246, 248, 249, 252, 254, 259, 260, 261, 263, 264, 268, 269, 270, 271, 272, 273, 274, 276, 279, 280, 281, 283, 288, 290, 291, 294, 295, 296, 299, 301, 302, 306, 307, 308, 309, 310, 312, 313, 316, 317, 318, 319, 321, 322, 326, 327, 329, 335, 336, 337, 341, 343, 348, 351, 353, 356, 358, 360, 361, 363, 366, 367, 368, 369, 372, 373, 374, 376, 377, 380, 381, 384, 385, 386], ['polo neck', 1, 2, 5, 6, 7, 10, 11, 16, 18, 19, 26, 33, 35, 3

In [18]:
colour_sorted= sorted(Dict_colour, key=Dict_colour.get, reverse=True)
for r in colour_sorted:
    print(r, Dict_colour[r])

black tshirt 52
multicolor tshirt 40
blue tshirt 30
green tshirt 30
white tshirt 22
yellow tshirt 19
dark blue tshirt 19
red tshirt 18
grey tshirt 13
brown tshirt 11
maroon tshirt 10
dark green tshirt 10
light blue tshirt 9
pink tshirt 7
orange tshirt 7
black white tshirt 5
black dark blue blue tshirt 4
white black tshirt 4
black grey tshirt 4
black yellow tshirt 3
black red tshirt 3
maroon black tshirt 3
red black tshirt 2
grey black tshirt 2
maroon grey tshirt 2
red white tshirt 2
pink black tshirt 2
green black tshirt 2
black dark blue grey tshirt 2
light green tshirt 2
white blue grey tshirt 2
white orange tshirt 2
beige tshirt 2
light blue red dark blue tshirt 2
black green white tshirt 2
white blue yellow tshirt 1
brown black tshirt 1
yellow black tshirt 1
dark blue white brown tshirt 1
white black pink tshirt 1
light blue white tshirt 1
dark blue black tshirt 1
white blue tshirt 1
black dark blue silver tshirt 1
yellow blue tshirt 1
light green black tshirt 1
black blue tshirt 1

In [19]:
row=colour_sorted[0:5]+colour_sorted[-5:]
with open('colour_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_colour[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['black tshirt', 14, 15, 18, 36, 37, 49, 52, 54, 56, 58, 60, 69, 88, 96, 108, 110, 120, 121, 126, 134, 140, 167, 169, 172, 184, 186, 188, 190, 191, 203, 211, 219, 226, 230, 231, 238, 245, 246, 249, 256, 259, 260, 263, 269, 274, 330, 344, 356, 361, 375, 380, 381], ['multicolor tshirt', 0, 6, 25, 34, 35, 38, 41, 67, 75, 79, 103, 105, 114, 119, 123, 124, 145, 164, 197, 199, 222, 227, 234, 236, 239, 244, 261, 273, 281, 286, 305, 312, 315, 316, 323, 340, 364, 374, 376, 377], ['blue tshirt', 5, 16, 23, 32, 42, 53, 78, 80, 101, 116, 133, 146, 153, 174, 178, 193, 195, 214, 220, 262, 270, 275, 288, 290, 300, 317, 321, 339, 348, 350], ['green tshirt', 10, 45, 47, 64, 91, 95, 99, 102, 104, 109, 130, 135, 138, 152, 175, 180, 204, 209, 212, 229, 250, 251, 252, 289, 302, 328, 337, 351, 362, 378], ['white tshirt', 100, 112, 147, 156, 157, 161, 198, 218, 235, 237, 254, 265, 294, 303, 306, 326, 338, 341, 363, 365, 369, 382], ['red white black tshirt', 347], ['maroon beige tshirt', 360], ['grey blue ts